In [ ]:
"""
ABM model: Configuration parameters

@author: hector@bith.net
"""
import sys

In [ ]:
class Config:
    T: int = 1000  # Time (1000)
    N: int = 100  # Number of firms (100)

    eta: float = 0.0001  # ŋ Inverse elasticity: ŋ=1/10000 -> perfect competition, ŋ=1/4 -> high market power

    # firms:                 balance sheet => K = A + L
    firms_K_i0: float = 5.0  # capital
    firms_A_i0: float = 1.0  # assets
    firms_L_i0: float = 4.0  # loans (from bank sector)
    phi: float = 1.0  # Φ capital productivity: constant in this model without R&D
    threshold_bankrupt = 0  # A < threshold_bankrupt and firm will fail
    g: float = 1.0  # cost per unit of capital
    w: float = 1.0  # markdown interest rate (the higher it is, the monopolistic power of bank sector)
    k: float = 1.0  # capital intensity rate K/N
    b: float = 1.0  # parameter of bankruptcy cost (b>0)
    beta: float = 0.02  # β skewness parameter -1 ... 1
    m: float = 0.03  # percentage of K that should be in cash

    # bank sector:                   balance sheet => L = A + D
    bank_sector_A_i0: float = 4*T*N  # L and D are set inside bank.py
    r_i0: float = 0.02  # initial rate of interest charged to firms by loans
    lambda_param: float = 0.3  # λ, to determine credit allotted for firms L=A/alfa, 0 < λ < 1
    alpha: float = 0.08  # α ratio equity/loan,  Ls=A/α

    # seed used:
    default_seed: int = 20579

    def __str__(self, separator=""):
        description = sys.argv[0] if not separator else ""
        for attr in dir(self):
            value = getattr(self, attr)
            if isinstance(value, int) or isinstance(value, float):
                description += f" {attr}={value}{separator}"
        return description

In [ ]:
"""
ABM model

@author: hector@bith.net
"""

In [ ]:
class BankSector:
    def __init__(self, its_model, A_i0=None):
        self.model = its_model
        self.bad_debt = 0.0
        self.profits = 0.0
        self.totalA = 0.0
        self.totalK = 0.0
        self.failures = 0
        self.A = A_i0 if A_i0 else self.model.config.bank_sector_A_i0
        self.L = self.determine_new_credit_suppy()
        self.D = self.determine_deposits()
        if self.D < 0:
            raise ValueError("error bank.D<0 due to " +
                             f"D=L-A={self.model.log.format(self.L)}-{self.model.log.format(self.A)}")
        self.estimate_total_a_k()

    def determine_deposits(self):
        # L = A + D, ----> D = L-A
        return self.L - self.A

    def determine_profits(self):
        # (Equation 34)
        profits_loans = 0.0
        for firm in self.model.firms:
            profits_loans += firm.r * firm.L
        remunerations_of_deposits_and_networth = self.determine_average_interest_rate() * (self.D + self.A)
        result = profits_loans - remunerations_of_deposits_and_networth
        self.model.log.debug(f"bank_sector profits={result} = profits_loans({profits_loans}) " +
                             f"- remuneration_deposits_and_assets({remunerations_of_deposits_and_networth})")
        return result

    def determine_average_interest_rate(self):
        # avg_r = sum(firm.r for firm in self.model.firms) / len(self.model.firms)
        # return avg_r if avg_r > self.model.config.r_i0 else self.model.config.r_i0
        return 0.01  # TODO

    def determine_net_worth(self):
        # (Equation 35) At = At-1 + profits - bad_debt
        net_worth = self.A + self.profits - self.bad_debt
        self.model.log.debug(f"bank_sector A={net_worth}={self.A}+{self.profits}-{self.bad_debt}")
        if net_worth < 0:
            # raise Exception(f"bank_sector failed at t={self.model.t+1} A=A + profits - bad_debt " +
            #                 f"--> {net_worth}={self.A}+{self.profits}-{self.bad_debt}")
            self.model.log.debug(f"bank_sector failed A=A_i0 ({self.model.config.bank_sector_A_i0})")
            net_worth = self.model.config.bank_sector_A_i0
        return net_worth

    def __str__(self):
        return f"bankSector L={self.model.log.format(self.L)} A={self.model.log.format(self.A)} " + \
            f"D={self.model.log.format(self.D)}"

    def determine_step_results(self):
        self.profits = self.determine_profits()
        self.A = self.determine_net_worth()
        self.L = self.determine_new_credit_suppy()
        self.D = self.determine_deposits()

    def initialize_step(self):
        self.bad_debt = 0
        self.failures = 0
        self.estimate_total_a_k()

    def determine_firm_capacity_loan(self, firm):
        # (Equation 11 of paper a new approach to business fluctuations)
        offeredL = (self.model.config.lambda_param * self.L * firm.K / self.totalK +
                    (1 - self.model.config.lambda_param) * self.L * firm.A / self.totalA)
        self.model.log.debug(f"bank_sector {firm} offeredL={offeredL}")
        return offeredL

    def determine_new_credit_suppy(self):
        credit_supply = self.A / self.model.config.alpha
        self.model.log.debug(f"bank_sector.L={credit_supply}")
        return credit_supply

    def add_bad_debt(self, firm):
        amount = firm.L - firm.K
        if amount < 0:
            self.model.log.debug(f"bank_sector {firm} fails and bad_debt increases in {amount}")
            self.bad_debt += -amount
        else:
            self.model.log.debug(f"bank_sector {firm} fails but no bad_debt is generated")
        self.failures += 1

    def estimate_total_a_k(self):
        self.totalA = sum(float(firm.A) for firm in self.model.firms)
        self.totalK = sum(float(firm.K) for firm in self.model.firms)
        self.model.log.debug(f"bank_sector Σ firm.A={self.totalA} Σ firm.K={self.totalK}")

In [ ]:
"""
ABM model

@author: hector@bith.net
"""
import random

In [ ]:
class Firm:
    def __init__(self, new_id=None, its_model=None):
        if its_model:
            self.id = new_id
            self.model = its_model
            self.failures = 0
        self.K = self.model.config.firms_K_i0
        self.A = self.model.config.firms_A_i0
        self.L = self.model.config.firms_L_i0
        self.r = self.model.config.r_i0
        self.gamma = (self.model.config.w / self.model.config.k) + (self.model.config.g * self.r)
        self.phi = self.model.config.phi
        self.pi = self.c = self.Y = self.u = self.gap_of_L = 0.0
        self.desiredK = self.demandL = self.offeredL = self.I = 0.0

    def __str__(self, short: bool = False):
        init = "firm#" if not short else "#"
        if self.failures > 0:
            return f"{init}{self.id}.{self.failures:<2}"
        else:
            return f"{init}{self.id}   "

    def do_step(self):
        self.gamma = self.determine_cost_per_unit_of_capital()
        self.desiredK = self.determine_desired_capital()
        self.I = self.determine_investment()
        self.demandL = self.determine_demand_loan()
        self.offeredL = self.model.bank_sector.determine_firm_capacity_loan(self)
        self.L = self.determine_new_loan()
        self.K = self.adjust_capital()
        self.r = self.determine_interest_rate()
        self.c = self.determine_marginal_operating_cost()
        self.Y = self.determine_output()  # not used in pi, substituted by phi*K
        self.u = self.determine_u()
        self.pi = self.determine_profits()
        self.A = self.determine_net_worth()
        if self.is_bankrupted():
            self.set_failed()
        else:
            self.K = self.adjust_capital()

    def determine_cost_per_unit_of_capital(self):
        # (Before equation 2)  gamma
        gamma = (self.model.config.w / self.model.config.k) + (self.model.config.g * self.r)
        self.model.log.debug(f"{self} γ={gamma}")
        return gamma

    def determine_marginal_operating_cost(self):
        # (Equation 2)
        c = self.gamma / self.phi
        self.model.log.debug(f"{self} c={c}")
        return c

    def determine_output(self):
        output = self.phi * self.desiredK
        self.model.log.debug(f"{self} Y={output}")
        return output

    def determine_interest_rate(self):
        # (Equation 33)
        rate = self.model.config.beta * self.L / self.A
        # if rate > 0.10:
        #     rate = 0.10
        # if rate < 0.01:
        #     rate = 0.01
        self.model.log.debug(f"{self} r={rate}")
        return rate

    def determine_desired_capital(self):
        # (Equation 22)
        desiredK = (1 - self.model.config.eta) ** 2 / (self.model.config.b * self.gamma) - \
                   (1 - self.model.config.eta) / (self.model.config.b * self.phi) + \
                   self.A / (2 * self.gamma)
        self.model.log.debug(f"{self} desiredK={desiredK}")
        return desiredK

    def determine_investment(self):
        # (Below equation 33)
        investment = self.desiredK - self.K
        self.model.log.debug(f"{self} I={investment}")
        return investment

    def determine_demand_loan(self):
        # (Over equation 33)
        demandL = self.L + (1 + self.model.config.m) * self.I - self.pi
        if demandL < 0:
            self.model.log.debug(f"{self} demandL<0 ({demandL}), so  demandL={self.L} (prev)")
            demandL = self.L
        else:
            self.model.log.debug(f"{self} demandL={demandL}")
        return demandL

    def determine_new_loan(self):
        if self.demandL > self.offeredL:
            self.gap_of_L = (self.demandL - self.offeredL)
            self.model.log.debug(f"{self} L=oL={self.offeredL} gapL={self.gap_of_L}")
            return self.offeredL
        else:
            self.gap_of_L = 0.0
            self.model.log.debug(f"{self} L=dL={self.demandL}")
            return self.demandL

    def determine_u(self):
        # stochastic demand [0,2]
        u = random.uniform(0, 2)
        self.model.log.debug(f"{self} u={u}")
        return u

    def determine_profits(self):
        # (Equation 24)  , but with Y = phi*K and simplifying
        profits = self.u * (self.model.config.eta + (1 - self.model.config.eta) * self.model.config.phi * self.K) - \
                  self.gamma * self.K
        self.model.log.debug(f"{self} π={profits}")
        return profits

    def determine_net_worth(self):
        # (Equation 8)
        new_A = self.A + self.pi
        self.model.log.debug(f"{self} A={new_A}")
        return new_A

    def is_bankrupted(self):
        return self.A < self.model.config.threshold_bankrupt

    def set_failed(self):
        self.model.bank_sector.add_bad_debt(self)
        self.failures += 1
        self.__init__()

    def adjust_capital(self):
        newK = self.A + self.L
        self.model.log.debug(f"{self} K={newK}")
        return newK

In [ ]:
"""
ABM model auxiliary file: logging facilities
@author: hector@bith.net
"""
import logging
import numpy as np
import sys

In [ ]:
class Log:
    """
    The class acts as a logger and helpers for the Model.
    """
    logger = logging.getLogger("model")
    OUTPUT_DIRECTORY = "output"
    model = None
    log_level = "ERROR"
    progress_bar = None
    what_keywords = []

    def __init__(self, model=None):
        if model:
            self.model = model
        else:
            self.model = MockedModel()
        self.colors = LogColors()

    def set_model(self, its_model, plot, num_model, multiple_models_will_be_run: False):
        if not self.what_keywords and its_model.log.what_keywords:
            self.what_keywords = its_model.log.what_keywords
        self.model = its_model
        if multiple_models_will_be_run:
            self.model.export_datafile = f"{self.OUTPUT_DIRECTORY}/model_{num_model}.txt"
            self.model.statistics.export_datafile = self.model.export_datafile
            self.model.statistics.interactive = False
            self.model.statistics.multiple = True
        else:
            if plot == PlotMethods.gretl and not self.model.export_datafile:
                self.model.export_datafile = f"{self.OUTPUT_DIRECTORY}/model.txt"
            self.model.statistics.interactive = True
            self.model.statistics.multiple = False
        self.model.log = self

    @staticmethod
    def format(number):
        if isinstance(number, int) or isinstance(number, np.int32):
            result = f"{number:4}"
        else:
            result = f"{number:7.3f}"
            while len(result) > 7 and result[-1] in "0":
                result = result[:-1]
            while len(result) > 7 and result.find('.') > 0:
                result = result[:-1]
        return result if result[-1] != '.' else f' {result[:-1]}'

    @staticmethod
    def get_level(option):
        try:
            return getattr(logging, option.upper())
        except AttributeError:
            logging.error(f" '--log' must contain a valid logging level and {option.upper()} is not.")
            sys.exit(-1)

    def debug(self, text, before_start=False):
        if text and not self.model.test:
            self.logger.debug(f"{self.__format_t__(before_start)} {text}")

    def info(self, text, before_start=False):
        if text and not self.model.test:
            self.logger.info(f" {self.__format_t__(before_start)} {text}")

    def warning(self, text, before_start=False):
        if text and not self.model.test:
            self.logger.warning(f" {self.__format_t__(before_start)} {text}")

    def error(self, text, before_start=False):
        if text:
            self.logger.error(self.colors.fail(f"{self.__format_t__(before_start)} {text}"))

    def __format_t__(self, before_start=False):
        return f"     {self.model.get_id(short=True)}" if before_start \
            else f"t={self.model.t + 1:03}{self.model.get_id(short=True)}"

    def define_log(self, log: str, logfile: str = '', what=[]):
        # noinspection SpellCheckingInspection
        formatter = logging.Formatter('%(levelname)s %(message)s')
        self.log_level = Log.get_level(log.upper())
        self.what_keywords = what
        self.logger.setLevel(self.log_level)
        if self.logger.hasHandlers():
            self.logger.handlers.clear()
        if logfile:
            if not logfile.startswith(self.OUTPUT_DIRECTORY):
                logfile = f"{self.OUTPUT_DIRECTORY}/{self.model.get_id_for_filename()}{logfile}"
            else:
                logfile = f"{self.model.get_id_for_filename()}{logfile}"
            fh = logging.FileHandler(logfile, 'a', 'utf-8')
            fh.setLevel(self.log_level)
            fh.setFormatter(formatter)
            self.logger.addHandler(fh)
        else:
            ch = logging.StreamHandler()
            ch.setLevel(self.log_level)
            ch.setFormatter(formatter)
            self.logger.addHandler(ch)

    def info_firm(self, firm, before_start=False):
        text = f"{firm.__str__()}  "
        if not before_start:
            if self.what_keywords and not self.model.test:
                for elem in self.model.statistics.data:
                    if elem in self.what_keywords and elem.startswith('firms'):
                        text += f" {elem.replace('firms_', '')}="
                        text += f"{self.format(self.model.statistics.data[elem].get_value(firm))}"
                self.info(text, before_start)

    def initialize_model(self):
        if self.logger.level == Log.get_level("ERROR") and not self.model.test:
            self.progress_bar = Bar(f"Executing {self.model.get_id()}", max=self.model.config.T)

    def step(self, log_info, before_start=False):
        if not self.model.test and self.model.statistics.interactive:
            if self.progress_bar:
                self.progress_bar.next()
            else:
                self.warning(log_info, before_start=before_start)
                self.model.statistics.info_status(before_start=before_start)

    def finish_model(self):
        if not self.model.test:
            if self.progress_bar:
                self.progress_bar.finish()
            else:
                self.info(f"finish: {self.model.get_id()} {self.model.model_title}" +
                          f"T={self.model.config.T} N={self.model.config.N}")

In [ ]:
class MockedModel:
    test = True
    model_title = ""
    before_start = True

    statistics = None
    log = None
    t = 0
    config = None

    def get_id_for_filename(self, _):
        return 0

    def get_id(self, _):
        return ""

In [ ]:
class LogColors:
    colors = True

    def warning(self, text):
        if self.colors:
            import colorama
            return colorama.Fore.YELLOW + text + colorama.Fore.RESET

    def fail(self, text):
        if self.colors:
            import colorama
            return colorama.Fore.RED + text + colorama.Fore.RESET

    def remark(self, text):
        if self.colors:
            import colorama
            return colorama.Style.BRIGHT + text + colorama.Style.NORMAL

    def __init__(self):
        try:
            import colorama
        except ImportError:
            self.colors = False
        else:
            colorama.init(autoreset=True)

In [ ]:
"""
ABM model auxiliary file: logging facilities
@author: hector@bith.net
"""
import numpy as np
import math
from enum import Enum

In [ ]:
# noinspection SpellCheckingInspection
class PlotMethods(str, Enum):
    pyplot = "pyplot"
    bokeh = "bokeh"
    grace = "grace"
    gretl = "gretl"
    screen = "screen"

    @classmethod
    def _missing_(cls, _):
        return cls.pyplot

    def plot(self, plot_min, plot_max, filename, title, y_label, series_name,
             data, model, multiple=None, multiple_key=None):
        match self.name:
            case PlotMethods.bokeh | PlotMethods.screen:
                import bokeh.plotting
                from bokeh.palettes import Category20
                p = bokeh.plotting.figure(title=title, x_axis_label="t", y_axis_label=y_label,
                                          sizing_mode="stretch_width", height=550)
                if not multiple:
                    xx, yy = StatsArray.get_plot_elements(data, plot_min, plot_max)
                    p.line(xx, yy, color="blue", line_width=2)
                else:
                    i = 0
                    for element in multiple:
                        xx, yy = StatsArray.get_plot_elements(multiple[element][multiple_key].data,
                                                              plot_min, plot_max)
                        p.line(xx, yy, color=Category20[20][i % 20], line_width=2, legend_label=element)
                        i += 1
                if self.name == PlotMethods.screen:
                    bokeh.plotting.output_notebook()
                    bokeh.plotting.show(p)
                else:
                    bokeh.plotting.output_file(filename + ".html", title=title)
                    bokeh.plotting.save(p)
                    return filename + ".html"

            case PlotMethods.grace:
                from pygrace.project import Project
                from pygrace.colors import ColorBrewerScheme

                if multiple:
                    plot = Project(colors=ColorBrewerScheme('Paired'))
                else:
                    plot = Project()

                graph = plot.add_graph()
                graph.title.text = title.encode('ascii', 'replace').decode()
                if multiple:
                    i = 0
                    datasets = []
                    for element in multiple:
                        datasets.append(
                            graph.add_dataset(StatsArray.get_plot_elements(multiple[element][multiple_key].data,
                                                                           plot_min, plot_max, two_list=False),
                                              legend=element))
                        datasets[-1].symbol.fill_color = i
                        i += 1
                else:
                    graph.add_dataset(StatsArray.get_plot_elements(data, plot_min, plot_max, two_list=False))
                    graph.yaxis.label.text = 'y_label'
                graph.autoscalex()
                graph.autoscaley()
                graph.autoticky()
                graph.xaxis.label.text = 't'
                plot.saveall(filename + ".agr")
                return filename + ".agr"

            case PlotMethods.gretl:
                with open(filename + ".inp", 'w', encoding="utf-8") as script:
                    import os
                    script.write(f"set workdir " + os.getcwd() + "\n")
                    script.write(f"open {model.export_datafile}\n")
                    script.write("setobs 1 1 --special-time-series\n")
                    if multiple:
                        series_to_plot = f" {series_name}_0"
                        for i in range(1, len(multiple)):
                            another_model_filename = model.export_datafile.replace("_0.txt", f"_{i}.txt")
                            script.write(f"append {another_model_filename}\n")
                            series_to_plot += f" {series_name}_{i}"
                        script.write(f"gnuplot {series_to_plot} --time-series --with-lines\n")
                    else:
                        if model.get_id_for_filename() != '':
                            series_name += "_" + model.get_id_for_filename().replace("_", "")
                        script.write(f"gnuplot {series_name} --time-series --with-lines\n")
                    script.write(f"exit()\n")
                return filename + ".inp"

            case _:
                import matplotlib.pyplot as plt
                plt.clf()
                xx = []
                if multiple:
                    for element in multiple:
                        xx, yy = StatsArray.get_plot_elements(multiple[element][multiple_key].data,
                                                              plot_min, plot_max)
                        plt.plot(xx, yy, label=element)
                    plt.legend()
                else:
                    xx, yy = StatsArray.get_plot_elements(data, plot_min, plot_max)
                    plt.plot(xx, yy)
                    plt.ylabel(y_label)
                plt.xticks(xx)
                plt.xlabel("t")
                plt.title(title)
                plt.savefig(filename + ".png")
                # plt.show()
                return filename + ".png"

    @staticmethod
    def check_sys_argv():
        import sys
        for i in range(len(sys.argv) - 1):
            if sys.argv[i] == "--plot" and sys.argv[i + 1].startswith("-"):
                sys.argv.insert(i + 1, PlotMethods('default').name)
        if sys.argv[-1] == '--plot':
            sys.argv.append(PlotMethods('default').name)

In [ ]:
class StatsArray:
    def __init__(self, its_model, data_type, description,
                 short_description, prepend="", plot=True, attr_name=None, logarithm=False, show=True):
        self.description = description
        self.short_description = short_description
        self.model = its_model
        self.prepend = prepend
        self.its_name = ""
        self.repr_function = ""
        self.show = show
        self.logarithm = logarithm
        if attr_name:
            self.attr_name = attr_name
        else:
            self.attr_name = self.short_description
        self.data = np.zeros(its_model.config.T, dtype=data_type)
        self.do_plot = plot

    def get_value(self, firm):
        element = getattr(firm, self.attr_name)
        if callable(element):
            return element()
        else:
            return element

    def __return_value_formatted__(self):
        result = f"{self.short_description}"
        result += "Ξ" if self.logarithm else "="
        result += f"{self.model.log.format(self.data[self.model.t])}"
        return result

    def __getitem__(self, t):
        return self.model.log.format(self.data[t])

    def __get__(self):
        return self.data

    def get_description(self):
        return f"{self.repr_function if self.repr_function else ' '} {self.attr_name:10}"

    def plot(self, plot_format: PlotMethods, plot_min: int = None, plot_max: int = None, multiple=None,
             multiple_key=None):
        if not plot_min or plot_min < 0:
            plot_min = 0
        if not plot_max or plot_max > self.model.config.T:
            plot_max = self.model.config.T
        if self.do_plot:
            y_label = self.repr_function + self.description + "(ln)" if self.logarithm else ""
            series_name = f"{self.its_name}{self.short_description.upper()}"
            if multiple:
                filename = self.model.statistics.OUTPUT_DIRECTORY + "/" + self.filename()
                title = self.its_name + " " + self.repr_function + self.description
            else:
                filename = self.model.statistics.OUTPUT_DIRECTORY + "/" + \
                           self.model.get_id_for_filename() + self.filename()
                title = self.its_name + " " + self.repr_function + self.description + self.model.model_title

            return plot_format.plot(plot_min, plot_max, filename, title, y_label, series_name, self.data,
                                    self.model, multiple, multiple_key)
        return None

    @staticmethod
    def get_plot_elements(the_array, plot_min, plot_max, two_list=True):
        xx = []
        yy = []
        for i in range(plot_min, plot_max):
            if not np.isnan(the_array[i]):
                if two_list:
                    xx.append(i)
                    yy.append(the_array[i])
                else:
                    xx.append((i, the_array[i]))
        if two_list:
            return xx, yy
        else:
            return xx

    def __str__(self):
        if self.its_name != "":
            return self.its_name + self.short_description.upper()
        else:
            return self.short_description.upper()

    def filename(self):
        return self.its_name.lower() + "_" + self.description.lower().replace(" ", "_")

In [ ]:
class StatsFirms(StatsArray):
    def __init__(self, its_model, data_type, description, short_description,
                 prepend="", plot=True, attr_name=None, function=sum, repr_function="Σ", logarithm=False, show=True):
        super().__init__(its_model, data_type, description, short_description,
                         prepend, plot, attr_name, logarithm, show)
        self.function = function
        self.its_name = "Firms"
        self.repr_function = repr_function

    def store_statistics(self):
        result = self.function(self.get_value(firm) for firm in self.model.firms)
        self.data[self.model.t] = math.log(result) if self.logarithm else result
        if self.show:
            return self.prepend + self.repr_function + self.__return_value_formatted__()
        else:
            return ""

In [ ]:
class StatsBankSector(StatsArray):
    def __init__(self, its_model, data_type, description, short_description,
                 prepend="", plot=True, attr_name=None, logarithm=False, show=True):
        super().__init__(its_model, data_type, description, short_description,
                         prepend, plot, attr_name, logarithm, show)
        self.function = None
        self.its_name = "Bank"
        self.repr_function = ""

    def store_statistics(self):
        result = getattr(self.model.bank_sector, self.attr_name)
        self.data[self.model.t] = math.log(result) if self.logarithm else result
        if self.show:
            return self.prepend + self.__return_value_formatted__()
        else:
            return ""

In [ ]:
"""
ABM model auxiliary file: to have statistics and plot
@author:  hector@bith.net
"""

In [ ]:
import os

In [ ]:
class Statistics:
    OUTPUT_DIRECTORY = "output"

    # This time the idea is to use pandas to store the statistics
    def __init__(self, its_model):
        self.model = its_model
        self.data = {}
        self.plot_min = 0
        self.multiple = False
        self.plot_max = None
        self.plot_what = []
        import os
        if not os.path.isdir(self.OUTPUT_DIRECTORY):
            os.mkdir(self.OUTPUT_DIRECTORY)
        self.export_datafile = None
        self.export_description = None
        self.do_plot = False
        self.interactive = True

    def info_status(self, before_start=False):
        for firm in self.model.firms:
            self.model.log.info_firm(firm, before_start=before_start)

    def current_status_save(self):
        # it returns also a string with the status
        result = ""
        for item in self.data:
            result += self.data[item].store_statistics()
        return result

    def add(self, what, name, prepend="", symbol=None, attr_name=None, number_type=float, function=sum,
            repr_function="Σ", plot=True, logarithm=False, show=True):
        if not attr_name:
            attr_name = name
        if not symbol:
            symbol = name.replace(" ", "_")
            if len(symbol) != len(name):
                symbol = symbol.lower()
        if not symbol.isascii():
            symbol = name
        if not callable(function):
            raise TypeError("function parameter should be a callable type")
        if what == BankSector:
            self.data["bank_" + name.replace(" ", "_")] = StatsBankSector(self.model, number_type, name, symbol,
                                                                          prepend=prepend, plot=plot,
                                                                          attr_name=attr_name, logarithm=logarithm,
                                                                          show=show)
        else:
            self.data["firms_" + name.replace(" ", "_")] = StatsFirms(self.model, number_type, name, symbol,
                                                                      prepend=prepend, function=function,
                                                                      repr_function=repr_function,
                                                                      plot=plot, attr_name=attr_name,
                                                                      logarithm=logarithm, show=show)

    def get_export_path(self, filename):
        if not filename.startswith(Statistics.OUTPUT_DIRECTORY):
            filename = f"{Statistics.OUTPUT_DIRECTORY}/{self.model.get_id_for_filename()}{filename}"
        else:
            filename = f"{self.model.get_id_for_filename()}{filename}"
        return filename if filename.endswith('.txt') else f"{filename}.txt"

    def export_data(self, export_datafile=None, export_description=None):
        if export_datafile:
            if self.interactive:
                progress_bar = Bar('Saving output in ' + self.get_export_path(export_datafile),
                                   max=self.model.config.T)
            else:
                progress_bar = None
            with (open(export_datafile, 'w', encoding="utf-8") as save_file):
                if export_description:
                    save_file.write(f"# {export_description}\n")
                else:
                    save_file.write(f"# {__name__} T={self.model.config.T} N={self.model.config.N}\n")
                header = " t"
                for item in self.data:
                    header += "\t"
                    header += f"{self.data[item].__str__()}"
                    if self.model.model_id:
                        header += f"_{self.model.model_id}"
                save_file.write(header + "\n")
                for i in range(self.model.config.T):
                    line = f"{i:3}"
                    for item in self.data:
                        line += "\t" + self.data[item][i]
                    save_file.write(line + "\n")
                    if progress_bar:
                        progress_bar.next()
            if progress_bar:
                progress_bar.finish()

    def plot(self, results_multiple=None):
        if self.do_plot:
            if self.plot_what:
                what_to_plot = 0
                for item in self.data:
                    if item in self.plot_what:
                        what_to_plot += 1
            else:
                what_to_plot = len(self.data)
            plotted_files = []
            text = f"Saving {self.do_plot} plots"
            if self.interactive:
                progress_bar = Bar(f"{text} in {self.OUTPUT_DIRECTORY}/{self.model.get_id_for_filename()}*",
                                   max=what_to_plot)
            else:
                progress_bar = None
            for item in self.data:
                if not self.plot_what or item in self.plot_what:
                    if results_multiple:
                        plotted_files.append(self.data[item].plot(plot_format=self.do_plot,
                                                                  plot_min=self.plot_min, plot_max=self.plot_max,
                                                                  multiple=results_multiple, multiple_key=item))
                    else:
                        plotted_files.append(self.data[item].plot(plot_format=self.do_plot,
                                                                  plot_min=self.plot_min, plot_max=self.plot_max))
                    if progress_bar:
                        progress_bar.next()

            if progress_bar:
                progress_bar.finish()
            if results_multiple or not self.model.statistics.multiple:
                self.execute_program(plotted_files)

    def execute_program(self, plot_format_array):
        if plot_format_array.count(None) > 0:
            plot_format_array.remove(None)
        if len(plot_format_array) == 1 and self.model.statistics.interactive:
            import configparser
            config = configparser.ConfigParser()
            config_file = 'market_power.config'
            config.read(config_file)
            file_extension = plot_format_array[0][-3:]
            if file_extension in config:
                if 'program' in config[file_extension]:
                    executable = config[file_extension]['program']
                    if executable.lower() == "default":
                        import os
                        if os.name == 'nt':
                            plot_format_array[0] = plot_format_array[0].replace('/', '\\')
                        os.startfile(plot_format_array[0], 'open')
                    else:
                        import subprocess
                        import os.path
                        if os.path.exists(executable):
                            subprocess.run([executable, plot_format_array[0]], stdout=subprocess.DEVNULL, shell=True)

    def get_what(self):
        print(self.model.log.colors.remark(f"\t{'name':20} Σ=summation ¯=average, Ξ=logarithm scale"))
        for item in self.data:
            print(f"\t{item:20} {self.data[item].get_description()}")

    def get_default_plot_method(self):
        return PlotMethods('default')

    def enable_plotting(self, plot_format: PlotMethods, plot_min: int = None, plot_max: int = None, plot_what: str = ""):
        self.do_plot = plot_format
        if plot_min and plot_min >= 0:
            self.plot_min = plot_min
        if plot_max and plot_max <= self.model.config.T:
            self.plot_max = plot_max
        self.plot_what = plot_what

    def initialize_model(self, export_datafile=None, export_description=None):
        self.export_datafile = export_datafile
        self.export_description = export_description
        if self.model.log.progress_bar and not self.do_plot and not self.export_datafile:
            # if no debug, and no output to file and no plots, then why you execute this?
            self.do_plot = self.get_default_plot_method()
            self.model.log.warning("--plot enabled due to lack of any output", before_start=True)
        if not self.model.test:
            self.model.log.step(self.current_status_save(), before_start=True)

    def finish_model(self, export_datafile=None, export_description=None):
        if not self.model.test:
            self.export_data(export_datafile=export_datafile, export_description=export_description)
            self.plot()

    def clear_output_dir(self):
        for file in [f for f in os.listdir(self.OUTPUT_DIRECTORY)]:
            if os.path.isfile(self.OUTPUT_DIRECTORY + "/" + file):
                self.model.log.warning(f"Removing {self.OUTPUT_DIRECTORY + '/' + file}", before_start=True)
                os.remove(self.OUTPUT_DIRECTORY + "/" + file)

In [ ]:
def mean(data):
    """ returns the mean of an array"""
    result = i = 0
    for i, x in enumerate(data):
        result += x
    return result / i

In [ ]:
def all_values(data):
    result = []
    for i in enumerate(data):
        result.append(i)
    return result

In [ ]:
"""
ABM model: It contains the firms and the logic to execute the simulation

@author: hector@bith.net
"""
import random
import statistics

In [ ]:
class Model:
    firms = []
    t: int = 0  # current value of time, t = 0..Model.config.T
    bank_sector: BankSector
    bankruptcies = []

    test = False  # it's true when we are inside a test
    log: Log = None
    statistics: Statistics = None
    config: Config = None
    export_datafile = None
    export_description = None
    model_id = ""
    model_title = ""

    def __init__(self, firm_class=Firm, model_id="", log=None, model_title="", **configuration):
        self.config = Config()
        if log:
            self.log = log
            log.model = self
        else:
            self.log = Log(self)
        self.model_id = model_id
        self.model_title = model_title
        self.firm_class = firm_class
        self.statistics = Statistics(self)
        if configuration:
            self.configure(**configuration)

    def configure(self, export_datafile=None, export_description=None, **configuration):
        for attribute in configuration:
            if hasattr(self.config, attribute):
                current_value = getattr(self.config, attribute)
                if isinstance(current_value, int):
                    setattr(self.config, attribute, int(configuration[attribute]))
                else:
                    if isinstance(current_value, float):
                        setattr(self.config, attribute, float(configuration[attribute]))
                    else:
                        raise Exception(f"type of config {attribute} not allowed: {type(current_value)}")
                match attribute:
                    case "N":
                        if self.config.N == 0:
                            raise ValueError("For config N values > 0")
                    case "T":
                        if self.config.T == 0:
                            raise ValueError("For config T values > 0")

                    case _:
                        pass
            else:
                raise LookupError(f"attribute '{attribute}' in config not found")
        self.initialize_model(export_datafile=export_datafile, export_description=export_description)

    def initialize_model(self, seed=None,
                         export_datafile=None, export_description=None):
        # what to plot and represent, and in which order
        self.statistics.add(what=Firm, name="K", prepend=" firms   ")
        self.statistics.add(what=Firm, name="A", prepend=" |")
        self.statistics.add(what=Firm, name="L", prepend=" ")
        self.statistics.add(what=Firm, name="profits", prepend=" ", symbol="π", attr_name="pi")
        self.statistics.add(what=Firm, name="Y", prepend=" ")
        self.statistics.add(what=Firm, name="r", prepend=" ", function=statistics.mean)
        self.statistics.add(what=Firm, name="I", prepend=" ")
        self.statistics.add(what=Firm, name="u", function=statistics.mean, repr_function="¯")
        self.statistics.add(what=Firm, name="desiredK", show=False)
        self.statistics.add(what=Firm, name="offeredL", show=False)
        self.statistics.add(what=Firm, name="demandL", show=False)
        self.statistics.add(what=BankSector, name="L", prepend="\n                bank    ")
        self.statistics.add(what=BankSector, name="A", prepend=" | ")
        self.statistics.add(what=BankSector, name="D", prepend="  ")
        self.statistics.add(what=BankSector, name="failures", symbol="fail", prepend=" ", number_type=int)
        self.statistics.add(what=BankSector, name="profits", symbol="π", prepend="  ", plot=False, attr_name="profits")
        self.statistics.add(what=BankSector, name="bad debt",
                            symbol="bd", prepend=" ", plot=False, attr_name="bad_debt")
        self.config.__init__()
        random.seed(seed if seed else self.config.default_seed)
        if export_datafile:
            self.export_datafile = export_datafile
        self.export_description = str(self.config) if export_description is None else export_description
        self.firms = []
        for i in range(self.config.N):
            self.firms.append(self.firm_class(new_id=i, its_model=self))
        self.bank_sector = BankSector(self)
        self.statistics.initialize_model(export_datafile=self.export_datafile,
                                         export_description=self.export_description)
        self.log.initialize_model()

    def do_step(self):
        self.bank_sector.initialize_step()
        for firm in self.firms:
            firm.do_step()
        self.bank_sector.determine_step_results()
        self.log.step(self.statistics.current_status_save())

    def finish_model(self):
        self.log.finish_model()
        self.statistics.finish_model(export_datafile=self.export_datafile, export_description=self.export_description)

    def run(self, export_datafile=None):
        self.initialize_model(export_datafile=export_datafile)
        for self.t in range(self.config.T):
            self.do_step()
        self.finish_model()
        return self.statistics.data, self.model_title

    def remove_failed_firms(self):
        for firm in self.firms:
            if firm.is_bankrupted():
                firm.set_failed()

    def get_id(self, short=False):
        if self.model_id:
            return f" model#{self.model_id}" if short else f"model #{self.model_id}"
        else:
            return "" if short else "model"

    def get_id_for_filename(self):
        if self.model_id:
            return f"{self.model_id}_"
        else:
            return ""

In [ ]:
"""
ABM model executor, to run interactively the models
@author: hector@bith.net
"""
import typer
from typing import List

In [ ]:
# noinspection SpellCheckingInspection
def run_interactive(config: List[str] = typer.Argument(None, help="Change config value (i.e. alpha=3.1, ? to list)"),
                    log: str = typer.Option(None, help="Log level messages (ERROR,WARNING,INFO,DEBUG)"),
                    logfile: str = typer.Option(None, help="File to send the logs to"),
                    log_what: str = typer.Option(None, help="What to log (apart from balances, ? to list)"),
                    save: str = typer.Option(None, help="Save the output of this execution"),
                    plot: PlotMethods = typer.Option(None, help="Save the plot (? to list formats)"),
                    plot_tmin: int = typer.Option(None, help="Min. time to represent in the plots"),
                    plot_tmax: int = typer.Option(None, help="Max. time to represent in the plots"),
                    plot_what: str = typer.Option("", help="What to plot (all if omitted, ? to list)"),
                    clear: bool = typer.Option(False, help="Clear the output folder before execute anything"),
                    n: int = typer.Option(Config.N, help="Number of firms"),
                    t: int = typer.Option(Config.T, help="Time repetitions")):
    logger = Log()
    models, title = manage_config_values(t, n, log, logfile, log_what, plot_tmin, plot_tmax,
                                         plot_what, plot, logger, config)
    if clear:
        do_clear_of_output_directory()
    results = {}
    for i in range(len(models)):
        logger.set_model(models[i], plot, i, len(models) != 1)
        result, name_of_result = run(models[i], save)
        results[name_of_result] = result
    if len(models) > 1 and (plot or plot_what):
        models[0].statistics.plot(results)

In [ ]:
def run_notebook():
    model = Model()
    # if we are running in a Notebook:
    model.statistics.enable_plotting(plot_format=PlotMethods.screen)
    model.statistics.interactive = False
    run(model)

In [ ]:
def do_clear_of_output_directory():
    model = Model()
    model.statistics.clear_output_dir()

In [ ]:
def run(model, save=None):
    return model.run(export_datafile=save)

In [ ]:
def check_what(logger, what, log_or_plot):
    result = []
    if what:
        mock_model = Model(log=logger)
        mock_model.test = True
        mock_model.initialize_model(export_datafile="mock")
        for item in what.split(","):
            if item not in mock_model.statistics.data:
                if item == '?':
                    if log_or_plot == 'plot':
                        print(logger.colors.remark(f"\t{'name':20} Σ=summation ¯=average, Ξ=logarithm scale"))
                    for valid_values in mock_model.statistics.data:
                        print(f"\t{valid_values:20} {mock_model.statistics.data[valid_values].get_description()}")
                else:
                    valid_values = {str(key) for key, value in mock_model.statistics.data.items()}
                    logger.error(f"{log_or_plot}_what must be one of {valid_values}", before_start=True)
                raise typer.Exit()
            else:
                result.append(item)
    return result

In [ ]:
def manage_plot_options(model, plot_tmin, plot_tmax, plot_what, plot, logger):
    plot_what = check_what(logger, plot_what, "plot")
    if (plot_tmin or plot_tmax or plot_what) and not plot:
        # if not enabled plot with a specific format, we assume the first type: pyplot
        plot = model.statistics.get_default_plot_method()
    if plot:
        model.statistics.enable_plotting(plot_format=plot, plot_min=plot_tmin,
                                         plot_max=plot_tmax, plot_what=plot_what)

In [ ]:
def manage_log_options(model, log, log_what, logfile, logger):
    log_what = check_what(logger, log_what, "log")
    if log_what and not log:
        log = "INFO"
    if not log:
        log = "ERROR"
    model.log.define_log(log=log, logfile=logfile, what=log_what)

In [ ]:
# noinspection SpellCheckingInspection
def manage_config_values(t, n, log, logfile, log_what, plot_tmin, plot_tmax, plot_what, plot, logger, config_list):
    params_only_present_once = {}
    params_present_multiple = {}
    mock_model = Model()
    if config_list:
        config_list.sort()
        for item in config_list:
            if item == '?':
                print(mock_model.config.__str__(separator="\n"))
                raise typer.Exit()
            try:
                name_config, value_config = item.split("=")
            except ValueError:
                logger.error("A config value for the model should be passed as parameter=value", before_start=True)
                raise typer.Exit(-1)
            try:
                getattr(mock_model.config, name_config)
            except AttributeError:
                logger.error(f"Configuration has no {name_config} parameter", before_start=True)
                raise typer.Exit(-1)
            try:
                setattr(mock_model.config, name_config, float(value_config))
                if name_config in params_only_present_once:
                    params_present_multiple[name_config] = [params_only_present_once[name_config], float(value_config)]
                    del params_only_present_once[name_config]
                else:
                    if name_config in params_present_multiple:
                        params_present_multiple[name_config].append(float(value_config))
                    else:
                        params_only_present_once[name_config] = float(value_config)
            except ValueError:
                logger.error(f"Value given for {value_config} is not valid", before_start=True)
                raise typer.Exit(-1)
    models = []
    num_combinations = 0
    combinations = []
    title = "" if num_combinations == 1 else f"{params_present_multiple}"
    for item in cartesian_product(params_present_multiple):
        combinations.append(item)
        num_combinations += 1
    for i in range(num_combinations):
        one_combination_of_multiple_params = combinations[i]
        if num_combinations > 1:
            model = Model(model_id=f"{i}", model_title=f"{one_combination_of_multiple_params}", log=logger)
        else:
            model = Model()
        if t != model.config.T:
            model.config.T = t
        if n != model.config.N:
            model.config.N = n
        manage_log_options(model, log, log_what, logfile, logger)
        manage_plot_options(model, plot_tmin, plot_tmax, plot_what, plot, logger)
        for param in params_only_present_once:
            setattr(model.config, param, params_only_present_once[param])
        for param in one_combination_of_multiple_params:
            setattr(model.config, param, one_combination_of_multiple_params[param])
        models.append(model)
    return models, title

In [ ]:
def cartesian_product(my_dictionary):
    from itertools import product
    return (dict(zip(my_dictionary.keys(), values)) for values in product(*my_dictionary.values()))

In [ ]:
def is_notebook():
    try:
        __IPYTHON__
        return get_ipython().__class__.__name__ != "SpyderShell"
    except NameError:
        return False

In [ ]:
if is_notebook():
    run_notebook()
else:
    if __name__ == "__main__":
        PlotMethods.check_sys_argv()
        typer.run(run_interactive)